In [1]:
import time

import pandas as pd

In [2]:
# 타이머 기능
def print_time(end, start):
    print('='*50)
    elapsed = end - start
    print('{} secs'.format(round(elapsed)))

In [3]:
st = time.time()

# 데이터 읽어오기
trn = pd.read_csv('../data/train_ver2.csv')

print_time(time.time(),st)

/home1/irteam/.pyenv/versions/3.6.4/envs/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


66 secs


In [10]:
st = time.time()

# 변수명 확인
print(len(trn.columns))
print(list(trn.columns))

print_time(time.time(),st)

48
['fecha_dato', 'ncodpers', 'ind_empleado', 'pais_residencia', 'sexo', 'age', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel', 'ult_fec_cli_1t', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'cod_prov', 'nomprov', 'ind_actividad_cliente', 'renta', 'segmento', 'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1', 'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1', 'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']
0 secs


In [5]:
st = time.time()

# 데이터 첫 10줄 미리보기
print(trn.head())

print_time(time.time(),st)

   fecha_dato  ncodpers ind_empleado pais_residencia sexo  age  fecha_alta  \
0  2015-01-28   1375586            N              ES    H   35  2015-01-12   
1  2015-01-28   1050611            N              ES    V   23  2012-08-10   
2  2015-01-28   1050612            N              ES    V   23  2012-08-10   
3  2015-01-28   1050613            N              ES    H   22  2012-08-10   
4  2015-01-28   1050614            N              ES    V   23  2012-08-10   

   ind_nuevo antiguedad  indrel       ...        ind_hip_fin_ult1  \
0        0.0          6     1.0       ...                       0   
1        0.0         35     1.0       ...                       0   
2        0.0         35     1.0       ...                       0   
3        0.0         35     1.0       ...                       0   
4        0.0         35     1.0       ...                       0   

  ind_plan_fin_ult1 ind_pres_fin_ult1 ind_reca_fin_ult1 ind_tjcr_fin_ult1  \
0                 0                 0  

In [11]:
st = time.time()

print(trn.info())

print_time(time.time(),st)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13647309 entries, 0 to 13647308
Data columns (total 48 columns):
fecha_dato               object
ncodpers                 int64
ind_empleado             object
pais_residencia          object
sexo                     object
age                      object
fecha_alta               object
ind_nuevo                float64
antiguedad               object
indrel                   float64
ult_fec_cli_1t           object
indrel_1mes              object
tiprel_1mes              object
indresi                  object
indext                   object
conyuemp                 object
canal_entrada            object
indfall                  object
tipodom                  float64
cod_prov                 float64
nomprov                  object
ind_actividad_cliente    float64
renta                    float64
segmento                 object
ind_ahor_fin_ult1        int64
ind_aval_fin_ult1        int64
ind_cco_fin_ult1         int64
ind_cder_fin_ult1  

In [13]:
trn['fecha_dato'].dtype

dtype('O')

In [14]:
trn['ind_recibo_ult1'].dtype

dtype('int64')

In [15]:
trn['ind_nom_pens_ult1'].dtype

dtype('float64')

In [19]:
st = time.time()

# observe numerical columns
num_cols = [col for col in trn.columns[:24] if trn[col].dtype in ['int64', 'float64']]
print(trn[num_cols].describe())

print_time(time.time(),st)

           ncodpers     ind_nuevo        indrel     tipodom      cod_prov  \
count  1.364731e+07  1.361958e+07  1.361958e+07  13619574.0  1.355372e+07   
mean   8.349042e+05  5.956184e-02  1.178399e+00         1.0  2.657147e+01   
std    4.315650e+05  2.366733e-01  4.177469e+00         0.0  1.278402e+01   
min    1.588900e+04  0.000000e+00  1.000000e+00         1.0  1.000000e+00   
25%    4.528130e+05  0.000000e+00  1.000000e+00         1.0  1.500000e+01   
50%    9.318930e+05  0.000000e+00  1.000000e+00         1.0  2.800000e+01   
75%    1.199286e+06  0.000000e+00  1.000000e+00         1.0  3.500000e+01   
max    1.553689e+06  1.000000e+00  9.900000e+01         1.0  5.200000e+01   

       ind_actividad_cliente         renta  
count           1.361958e+07  1.085293e+07  
mean            4.578105e-01  1.342543e+05  
std             4.982169e-01  2.306202e+05  
min             0.000000e+00  1.202730e+03  
25%             0.000000e+00  6.871098e+04  
50%             0.000000e+00  1.0185

In [21]:
st = time.time()

# observe numerical columns
cat_cols = [col for col in trn.columns[:24] if trn[col].dtype in ['O']]
print(trn[cat_cols].describe())

print_time(time.time(),st)

        fecha_dato ind_empleado pais_residencia      sexo       age  \
count     13647309     13619575        13619575  13619505  13647309   
unique          17            5             118         2       235   
top     2016-05-28            N              ES         V        23   
freq        931453     13610977        13553710   7424252    542682   

        fecha_alta  antiguedad ult_fec_cli_1t  indrel_1mes tiprel_1mes  \
count     13619575    13647309          24793   13497528.0    13497528   
unique        6756         507            223         13.0           5   
top     2014-07-28           0     2015-12-24          1.0           I   
freq         57389      134335            763    7277607.0     7304875   

         indresi    indext conyuemp canal_entrada   indfall   nomprov  \
count   13619575  13619575     1808      13461183  13619575  13553718   
unique         2         2        2           162         2        52   
top            S         N        N           KHE     